In [1]:
import cv2
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import numpy as np
from scipy.signal import butter, lfilter, find_peaks
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.fft import fft
from tensorflow.keras.models import load_model

In [3]:
foreheads = load_model('forehead.h5')

# Heart-Rate measurement 

In [4]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 30)
foreheads = load_model('forehead.h5')

filter_order = 4
low_cutoff = 0.7
high_cutoff = 4.0

buffer_size = 100
color_buffer = np.zeros(buffer_size)
sample_rate = 30.0
time_vector = np.arange(buffer_size) / sample_rate
update = 0
total = 0
size = 30
h , w = size,size
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb,(120, 120))
    
    yhat = foreheads.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    if yhat[0] == 1:
        x1 = sample_coords[0] * 640 
        y1 = sample_coords[1] * 480
        x2 = sample_coords[2] * 640
        y2 = sample_coords[3] * 480
        
        x = int((x1 + x2) / 2)
        y = int((y1 + y2) / 2)
        
        
        colors = frame[y-int(size/2):y+int(size/2),x-int(size/2):x+int(size/2),]
        cv2.rectangle(frame, (x-w//2, y-h//2), (x+w//2, y+h//2), (0, 255, 0), 2)
        roi = (x, y, size, size) 

        roi_frame = frame[y-int(size/2):y+int(size/2),x-int(size/2):x+int(size/2)]
        roi_gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
        b, a = butter(filter_order, [low_cutoff/sample_rate*2, high_cutoff/sample_rate*2], btype='bandpass')
        roi_filtered = lfilter(b, a, roi_gray)
        

        color_buffer = np.roll(color_buffer, -1)
        color_buffer[-1] = np.mean(roi_filtered)
        signal = color_buffer - np.mean(color_buffer)
        peaks, _ = find_peaks(signal, height=0)
        if len(peaks) >= 2:
            bpm = 70.0 / np.mean(np.diff(peaks)) * (buffer_size / sample_rate) 
            total += bpm
        else:
            bpm = 0.0
        
        
        if (update%20)==0:
            val = total/20
            total = 0

        frame[y-int(size/2):y+int(size/2),x-int(size/2):x+int(size/2),:] = np.zeros((size,size,3))
    else:
        val = 0
    
    cv2.putText(frame, 'Heart rate: {:.1f} bpm'.format(val), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    update+=1

    cv2.imshow('Video', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 126ms/step


# Heart-Rate measurement in lowLight

In [6]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 30)
foreheads = load_model('forehead.h5')

filter_order = 4
low_cutoff = 0.7
high_cutoff = 4.0

buffer_size = 100
color_buffer = np.zeros(buffer_size)

sample_rate = 30.0
time_vector = np.arange(buffer_size) / sample_rate
update = 0
total = 0
size = 30
h , w = size,size
while True:
    ret, frame = cap.read()
    if not ret:
        break
    

    r = frame[:,:,2]
    g = frame[:,:,1]
    b = frame[:,:,0]
    equalizedr =  cv2.equalizeHist(r)
    equalizedg =  cv2.equalizeHist(g)
    equalizedb =  cv2.equalizeHist(b)
    frame = cv2.merge([equalizedb,equalizedg,equalizedr])
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb,(120, 120))
    
    yhat = foreheads.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]

    x1 = sample_coords[0] * 640 
    y1 = sample_coords[1] * 480
    x2 = sample_coords[2] * 640
    y2 = sample_coords[3] * 480
    
    x = int((x1 + x2) / 2)
    y = int((y1 + y2) / 2)
    
    
    colors = frame[y-int(size/2):y+int(size/2),x-int(size/2):x+int(size/2),]
    cv2.rectangle(frame, (x-w//2, y-h//2), (x+w//2, y+h//2), (0, 255, 0), 2)
    roi = (x, y, size, size) 

    roi_frame = frame[y-int(size/2):y+int(size/2),x-int(size/2):x+int(size/2)]
    roi_gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
    b, a = butter(filter_order, [low_cutoff/sample_rate*2, high_cutoff/sample_rate*2], btype='bandpass')
    roi_filtered = lfilter(b, a, roi_gray)
    
    color_buffer = np.roll(color_buffer, -1)
    color_buffer[-1] = np.mean(roi_filtered)
    
    signal = color_buffer - np.mean(color_buffer)
    peaks, _ = find_peaks(signal, height=0)
    if len(peaks) >= 2:
        bpm = 70.0 / np.mean(np.diff(peaks)) * (buffer_size / sample_rate) 
        total += bpm
    else:
        bpm = 0.0
    
    
    if (update%20)==0:
        val = total/20
        total = 0

    frame[y-int(size/2):y+int(size/2),x-int(size/2):x+int(size/2),:] = np.zeros((size,size,3))

    cv2.putText(frame, 'Heart rate: {:.1f} bpm'.format(val), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    update+=1
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 112ms/step
